## Library import

In [43]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
import re

## Récupération du fichier html

Ce premier site est scrappable

In [7]:
url = "https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html"

response = requests.get(url=url)

print(response.status_code)

200


Celui ci ne l'est pas

In [8]:
url_amazon = "https://www.amazon.com"
response_amazon = requests.get(url_amazon)
print(response_amazon.status_code) 

503


In [9]:
soup = BeautifulSoup(response.text)
soup


<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    Philosophy | 
     Books to Scrape - Sandbox

</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="
    
" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="../../../../static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="../../../../static/oscar/css/styles.css" rel="stylesh

## Extraction des données

On observe que toutes les infos sont dans la balise article

In [11]:
articles = soup.find_all('article')

In [57]:
## prix du livre 
prix = []
## titre du livre 
titre = []
## 1 si le livre est en stock, 0 sinon 
bool_stock = []



for article in articles : 
    # on sélectionne la balise h3
    h3  = article.find('h3')
    ## au sein de h3, on sélectionne la balise a 
    a_tag = h3.find('a')
    ## au sein de a, on récupère le tag "title"
    title_name = a_tag["title"]
    titre.append(title_name)

    ## prix et diponibilité
    # prix
    div_price = article.find(class_="product_price")
    p_price = div_price.find(class_="price_color") 
    prix.append(p_price.get_text())

    #dispo 
    p_dispo = div_price.find(class_="instock availability")
    bool_stock.append(p_dispo.get_text().strip())

In [58]:
index=[i for i in range(len(prix))]
df = pd.DataFrame({"Titre" : titre,"Prix" : prix, "Stock" : bool_stock}, index)
df

,Titre,Prix,Stock
0,Sophie's World,Â£15.94,In stock
1,The Death of Humanity: and the Case for Life,Â£58.11,In stock
2,The Stranger,Â£17.44,In stock
3,Proofs of God: Classical Arguments from Tertul...,Â£54.21,In stock
4,Kierkegaard: A Christian Missionary to Christians,Â£47.13,In stock
5,"At The Existentialist CafÃ©: Freedom, Being, a...",Â£29.93,In stock
6,Critique of Pure Reason,Â£20.75,In stock
7,"Run, Spot, Run: The Ethics of Keeping Pets",Â£20.02,In stock
8,The Nicomachean Ethics,Â£36.34,In stock
9,Meditations,Â£25.89,In stock


Fonction pour clean les valeurs prix

In [55]:
def returns_only_number(price):
    price = re.findall(r'\d+\.\d+', price)
    price_value = float(price[0]) if price else None
    return price_value

In [59]:
df["Prix"] = df["Prix"].apply(returns_only_number)
df

,Titre,Prix,Stock
0,Sophie's World,15.94,In stock
1,The Death of Humanity: and the Case for Life,58.11,In stock
2,The Stranger,17.44,In stock
3,Proofs of God: Classical Arguments from Tertul...,54.21,In stock
4,Kierkegaard: A Christian Missionary to Christians,47.13,In stock
5,"At The Existentialist CafÃ©: Freedom, Being, a...",29.93,In stock
6,Critique of Pure Reason,20.75,In stock
7,"Run, Spot, Run: The Ethics of Keeping Pets",20.02,In stock
8,The Nicomachean Ethics,36.34,In stock
9,Meditations,25.89,In stock
